In [1]:
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import os
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large as deeplab

In [2]:
fd = "/home/jupyter/ai_font/data/exp0717/train_seg/raw_assembled"

In [3]:
class myDS(Dataset):
    def __init__(self, fd, train=True):
        self.path_list = []
        for root, dirs, files in os.walk(fd):
            for file in files:
                if file.endswith('.png') & ("checkpoint" not in file):
                    image_path = os.path.join(root, file)
                    label_root = root.replace("raw_assembled","raw_label")
                    label_file = file.replace(".png",".npy")
                    label_path = os.path.join(label_root, label_file)
                    self.path_list.append((image_path,label_path,))
        self.transforms = transforms.Compose([
                transforms.Resize(520),
                transforms.ToTensor(),  # Rescales to [0.0, 1.0]
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        self.label_transforms = transforms.Resize(520)
        
        if train:
            self.path_list = [p for i, p in enumerate(sorted(self.path_list)) if (i % 100) != 0]
        else:
            self.path_list = [p for i, p in enumerate(sorted(self.path_list)) if (i % 100) == 0]

    def __len__(self):
        return len(self.path_list)
    
    def __getitem__(self, idx):
        image_path, label_path = self.path_list[idx]
        image = Image.open(image_path).convert("RGB")
        mask = np.load(label_path)
        
        return self.transforms(image), self.label_transforms(torch.from_numpy(mask))

In [4]:
def calculate_accuracy(preds, targets):

    # Convert predictions to class labels by taking the argmax along the class dimension
    pred_labels = torch.argmax(preds, dim=1)
    target_labels = torch.argmax(targets, dim=1)
    
    # Check if the predicted labels are equal to the target labels
    correct = (pred_labels == target_labels).float()
    
    # Calculate the accuracy
    accuracy = correct.sum() / correct.numel()
    
    return accuracy.item() * 100  # Convert to percentage

In [5]:
ds = myDS("/home/jupyter/ai_font/data/exp0717/train_seg/raw_assembled", train=True)
dl = DataLoader(ds, batch_size=32, shuffle=True)
testds = myDS("/home/jupyter/ai_font/data/exp0717/train_seg/raw_assembled", train=False)
testdl = DataLoader(testds, batch_size=32, shuffle=False)

In [6]:
model = deeplab(weights_backbone="DEFAULT", num_classes=4)
model = model.cuda()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [8]:
epoch = 100

In [ ]:
pbar = tqdm(range(epoch))
for i in pbar:
    train_acc = []
    test_acc = []
    model.train()
    for j, data in enumerate(dl):
        optimizer.zero_grad()
        x, y = data
        x = x.cuda()
        y = y.cuda()
        pred = model(x)
        loss = criterion(pred['out'], y)
        loss.backward()
        optimizer.step()
        train_acc.append(calculate_accuracy(pred['out'], y))
        pbar.set_postfix(train=f"{j}/{len(dl)}")
    model.eval()
    for j, data in enumerate(testdl):
        x, y = data
        x = x.cuda()
        y = y.cuda()
        pred = model(x)
        test_acc.append(calculate_accuracy(pred['out'], y))
        pbar.set_postfix(test=f"{j}/{len(testdl)}")
    print(f"{i}: {np.mean(train_acc)} / {np.mean(test_acc)}")
    torch.save(model.state_dict(), f"model_{i}.pth")

  1%|          | 1/100 [17:18<28:33:22, 1038.41s/it, test=10/11]

0: 97.45128085344425 / 98.91342520713806


  2%|▏         | 2/100 [34:20<28:00:17, 1028.75s/it, test=10/11]     

1: 99.07898655550316 / 99.15885979479009


  3%|▎         | 3/100 [51:41<27:52:22, 1034.46s/it, test=10/11]     

...

 75%|███████▌  | 75/100 [21:35:05<7:11:07, 1034.72s/it, test=10/11]     

74: 99.53760905759391 / 99.44701248949224


 77%|███████▋  | 77/100 [22:09:47<6:37:49, 1037.79s/it, test=10/11]     

76: 99.53744548636311 / 99.14368391036987


 77%|███████▋  | 77/100 [22:20:10<6:37:49, 1037.79s/it, train=651/1082]